# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [3]:
import pandas as pd

In [20]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [19]:
# # Вариант 1
ratings = ratings[ (ratings['userId'] == 1) ]
ratings.groupby(ratings['rating']).count()[['userId']].head().reset_index()

# # Вариант 2
ratings[ratings.userId == 1].rating.value_counts()


Как это сделать для всех пользователей сразу:

In [21]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


In [34]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).reset_index().sort_values(by=[5.0], ascending=False).head(1)

rating,userId,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
671,All,1101,3326,1687,7271,4449,20064,10538,28750,7723,15095,100004


### Упражнение
Какой пользователь выставил больше всех пятерок?

In [33]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).sort_values(by=[5.0], ascending=False).head(1)

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
564,0,152,0,187,0,414,0,707,0,408


### Фильтры и вычисления с помощью loc и iloc

In [91]:
log = pd.read_csv('visit_log.csv', sep=';')
log.url.apply(lambda x: x[16:]).value_counts().head()

3c19b4ef7371864fa3    541
108ce4b365afb7b88e    534
23c64743ad842b9311    532
1afc5618562365020a    529
2fcb223266880ccf57    526
Name: url, dtype: int64

### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [42]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

,user_id,region
0,b1613cc09f,Russia
1,4c3ec14bee,Russia
2,a8c40697fb,Russia
3,521ac1d6a0,Russia
4,d7323c571c,Russia


In [43]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

,visit_id,region
0,e3b0c44298,Russia
1,6e340b9cff,Russia
2,96a296d224,Russia
3,709e80c884,Russia
4,df3f619804,Russia


In [ ]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

In [44]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2


In [54]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT,traffic_log,tcource_tupe,source_tupe
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2,organic,organic,ad
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2,NaN,NaN,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2,organic,organic,ad
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2,organic,organic,ad
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2,organic,organic,ad
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2,organic,organic,ad
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2,NaN,NaN,NaN
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2,NaN,NaN,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2,NaN,NaN,NaN
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,1.2,NaN,NaN,NaN


### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [94]:
log.loc[(log.traffic_source.isin(['yandex','google']), 'source_tupe')] ='organic'
log.loc[(log.traffic_source.isin(['paid','email']) & (log.region == 'Russia'), 'source_tupe')] ='ad'
log.loc[(log.traffic_source.isin(['paid','email']) & (log.region != 'Russia'), 'source_tupe')] ='other'
log.loc[(log.traffic_source != 'yandex') & (log.traffic_source != 'google') & (log.traffic_source != 'email') & (log.traffic_source != 'paid'), 'source_tupe'] =log['traffic_source']

log.head(20)


,timestamp,visit_id,url,region,user_id,traffic_source,source_tupe
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

In [ ]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

### Дата и время в pandas

In [ ]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

In [ ]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

In [ ]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [ ]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

### Методы работы со строками

In [ ]:
stats = pd.read_csv('keywords.csv')
stats.head()

Проверка наличия подстроки в строке в питоне:

In [ ]:
'охотник' in 'каждый охотник желает знать...'

Аналог в pandas:

In [ ]:
stats[stats.keyword.str.contains('охотник')].head()

[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [ ]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

### replace

In [ ]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

Аналог в pandas на запросах про сериалы:

In [ ]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

In [ ]:
serial.keyword.str.replace('сериалы', 'книги').head()

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [100]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pymystem3 import Mystem

In [118]:
search = 'курс гривны к рублю рубли рублях'

In [120]:
m = Mystem()
lemmas = m.lemmatize(search)


lemmas

['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 '\n']

In [121]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль \n'

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [5]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

# m = Mystem()
# data[data.keyword.lemmatize(str(data))].head()

data['keyword'][0]

NameError: name 'pd' is not defined

In [4]:

m = Mystem()
for text in data['keyword']:  
    lemmas = m.lemmatize(text)
print(lemmas)




NameError: name 'data' is not defined